#### __PrePreProcessing:__
Go over all participants<br>
for each part<br>
-   load 000 folder to Pupil Player<br>
-   cut video<br>
-   cut to frames<br>
-   find first appearince frame of 1st/2nd image
-   Add gaze csv starting time of said image. 
-   add to event.csv?

#### __PreProcessing:__
Go over participants<br>
for each part<br>
-   read event file<br> 
-   read time column<br>
-   clac deltas<br>
-   add gaze csv starting time to all deltas<br>
-   for each time:<br>
    -  go to csv, grab line with closest time <br>
    -  add frame number to output array<br>
    -  add time stamp to output array<br>

The table for each participant:<br>
key:    image number<br>
value:  list of norm_XY points<br> 
-  starts: calced start frame + 10. 
-  ends: starts + 35

In [48]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import pandas as pd
import csv
from enum import Enum


In [57]:
#orig included SG & VT 
# TODO TL
part_list = ["AR",  "ASH", "AT",  "DI",  "DM",  "DZ",
             "EL",  "EMZ", "HCH", "IL",  "IS",  "MA",
             "MBA", "NC",  "NCH", "NN",  "NS",  "NT", 
             "OA",  "OB",  "OR",  "RM",  "RN",  "SB", 
             "SHB", "SHL", "SHM", "SHO", "SHS", "SSW",
             "TCH", "TL",  "TLZ", "VT",  "YCH", "YH",  
             "YK",  "YL",  "YLA", "YM",  "YOB", "YUC"]
print(len(part_list))
gaze_dict_frame = {'AR': 153,  'ASH': 198, 'AT': 176,  'DI': 204,  'DM': 181,  'DZ': 199, 
                   'EL': 198,  'EMZ': 188, 'HCH': 200, 'IL': 198,  'IS': 201,  'MA': 202,
                   'MBA': 200, 'NC': 200,  'NCH': 193, 'NN': 143,  'NS': 159,  'NT': 144, 
                   'OA': 200,  'OB': 188,  'OR': 198,  'RM': 201,  'RN': 201,  'SB': 152, 
                   'SHB': 200, 'SHL': 141, 'SHM': 197, 'SHO': 200, 'SHS': 195, 'SSW': 201,  
                   'TCH': 198, 'TL': 197,  'TLZ': 199, 'YCH': 190, 'YH': 200, 
                   'YK': 182,  'YL': 194,  'YLA': 190, 'YM': 201,  'YOB': 196, 'YUC': 163 }

gaze_dict = {'AR':  105758.16850900001,  'ASH':  152858.473252,      'AT':  228.56323700000002, 
             'DI':  3600.981071,         'DM':  599030.1752385,      'DZ':  58244.5086345, 
             'EL':  101128.250285,       'EMZ':  7899.878946,        'HCH':  433614.166385, 
             'IL':  2212.143912,         'IS':  361283.802529,       'MA':  104314.0957885, 
             'MBA':  695783.4413000001,  'NC':  70050.155166,        'NCH':  489096.19665049994, 
             'NN':  141862.759811,       'NS':  358648.84771799995,  'NT':  573774.3492299999, 
             'OA':  12177.07619,         'OB':  1878.548347,         'OR':  430385.3604455, 
             'RM':  702233.1401905001,   'RN':  27537.832969,        'SB':  687314.6088785, 
             'SHB':  8271.210029000002,  'SHL':  174236.89568599997, 'SHM':  13931.109905000001, 
             'SHO':  53855.344588,       'SHS':  7132.935514499999,  'SSW':  707181.3576539999, 
             'TCH':  83899.5004785,      'TL':  51996.442503,        'TLZ':  5022.960182999999, 
             'YCH':  82085.2907705,      'YH':  -93731.5886265,      'YK':  107509.14350050001, 
             'YL':  -88811.536941,       'YLA':  22809.3528865,      'YM':  14567.499456000001, 
             'YOB':  177970.247576,      'YUC':  584142.682596 }

TIMESTAMP = 0
WORLD_IDX = 1
NORM_POS_X = 3
NORM_POS_Y = 4

42


In [ ]:
# copy outputs to monster

def copy_files_from_latest_folder(source, destination):

    directories = [d for d in os.listdir(source) if os.path.isdir(os.path.join(source, d))]
    
    # Get the latest directory based on modification time
    latest_folder = max(directories, key=lambda d: os.path.getmtime(os.path.join(source, d)))

    # Create the destination directory if it doesn't exist
    if not os.path.exists(destination):
        os.makedirs(destination)
    
    files_to_copy = ["gaze_positions.csv", "world.mp4"]

    for file_name in files_to_copy:
        source_file_path = os.path.join(source, latest_folder, file_name)
        destination_file_path = os.path.join(destination, file_name)
        shutil.copy2(source_file_path, destination_file_path)
        print(f"'{file_name}' copied from '{source}/{latest_folder}' to '{destination}'.")
    
def copy_relevant_outputs(part_list, root_source, root_destination):
    for part in part_list:
        
        folder_name = "" if (part == "RB") else "002" if (part == "NC") else "000"
        source = Path().joinpath(root_source, part, folder_name, "exports").resolve()
        destination = Path().joinpath(root_destination, part).resolve()
        copy_files_from_latest_folder(source, destination)

def copyOutToMonster():
    part_list = []

    root_source = Path("X:\Raw Data - From other computer\Sasha and Or\Sasha and Or")
    assert root_source.is_dir(), f"NOPE"

    root_destination = Path("Y:\mine_data")
    assert root_destination.is_dir(), f"NOPE"

    copy_relevant_outputs(part_list, root_source, root_destination)

In [ ]:
# Create script for ffmpeg

# # Call:
# root_destination = Path("Y:\mine_data")
# ffmpeg_script_path = "./ffmpeg_script"
# create_ffmpeg_script(part_list, ffmpeg_script_path, root_destination)

def write_to_file(file_path, content):
    try:
        with open(file_path, 'a') as file: # a for appand
            # Write content to the file
            file.write(content)
    except Exception as e:
        print(f"An error occurred: {e}")

def create_ffmpeg_script(part_list, ffmpeg_script_path, parts_dir):
    ffmpeg_script = Path(ffmpeg_script_path)
    for part in part_list:
        part_dir = Path().joinpath(parts_dir, part).resolve()
        mkdir_cm = f"\nmkdir ./{part_dir}/frames "
        ffmpeg_cm = f'\nffmpeg -i "./{part_dir}/world.mp4" -vsync passthrough "./{part_dir}/frames/frame%06d.png"'
        # write_to_file(ffmpeg_script, mkdir_cm)
        write_to_file(ffmpeg_script, ffmpeg_cm)

In [ ]:
def get_csv_column(csv_path, column_index, skip_lines = 0):
    
    col_vals = []

    with open(csv_path, mode='r', newline='') as csv_file:
        
        csv_reader = csv.reader(csv_file)
        
        for _ in range(skip_lines):
            next(csv_reader)

        for row in csv_reader:
            if len(row) > column_index:
                col_vals.append(row[column_index])
            else:
                # Handle case where row is shorter than expected
                print(f"Warning: Skipping row {row}, column index {column_index} out of range.")
            
        # col_vals = [row[column_index] for row in csv_reader]
        return col_vals

def get_images_order(events_csv):
    return get_csv_column(events_csv, 2, 2)

def binary_search(sorted_list, frame):
    # Binary search to find the index of the smallest element greater than or equal to x
    left, right = 0, len(sorted_list) - 1
    result = -1  # Initialize the result variable

    while left <= right:
        mid = left + (right - left) // 2
        gaze_mid = int(sorted_list[mid])
        if gaze_mid == frame:
            result = mid
            right = mid - 1  # Move to the left subarray to find the first instance
        elif gaze_mid < frame:
            left = mid + 1
        else:
            right = mid - 1
    return result

def find_ts_of_frame():

    for part, frame in gaze_dict_frame.items():

        gaze_csv = "./parts/breakItDown/" + part + "/gaze_positions.csv"
        gaze_ts = get_csv_column(gaze_csv, TIMESTAMP, skip_lines = 1)
        gaze_frames = get_csv_column(gaze_csv, WORLD_IDX, skip_lines = 1)
        if (len(gaze_ts) != len(gaze_frames)):
            print("ERROR")
            continue

        result = binary_search(gaze_frames, frame)   
            
        if result < 0:
            sec_result = binary_search(gaze_frames, (frame+1))
            if sec_result<0: print("error", part)
            result = sec_result

        ts = gaze_ts[result]
        print("'",part,"': ",ts,",")
 
def get_events_timestamps(events_csv):
    return get_csv_column(events_csv, 0, 2)

def calc_deltas(timestamps, factor = 0.0000001):
    start_ts = int(timestamps[0]) * factor
    deltas = [(float(ts) * factor - start_ts) for ts in timestamps]
    return deltas[0:-1]

def write_lists_to_csv(output_path, lists):
    # Open the CSV file for writing
    with open(output_path, 'w', newline='') as csvfile:
        # Create a CSV writer
        csv_writer = csv.writer(csvfile)

        # Write the header if needed
        # csv_writer.writerow(['Column1', 'Column2', ...])  # Uncomment and replace with your column names if needed

        # Write the rows to the CSV file
        csv_writer.writerows(lists)

def find_closest_ts(sorted_list, ts, low = 0):

    # Binary search to find the index of the smallest element greater than or equal to x
    high = len(sorted_list) - 1

    while low <= high:
        mid = (low + high) // 2

        if float(sorted_list[mid]) <= ts:
            low = mid + 1
        else:
            high = mid - 1

    # The index 'low' is the position where the smallest element greater than ts would be
    if low < len(sorted_list):
        return low
    else:
        # If x is greater than or equal to all elements in the list, return the last element
        return (len(sorted_list)-1)
    
def calc_real_ts_for_image(gaze_csv, images_ts):

    gaze_ts = get_csv_column(gaze_csv, TIMESTAMP, skip_lines = 1)
    real_ts_for_image = []
    real_idx_for_image = []
    low = 0

    for image_ts in images_ts:
        ts_idx = find_closest_ts(gaze_ts, image_ts, low)
        # ts_line_in_gaze_csv = ts_idx + 2
        # +2 cause (1)the first line is titles and (2)lines count starts from 1
        low = ts_idx
        real_idx_for_image.append(ts_idx)
        real_ts_for_image.append(gaze_ts[int(ts_idx)])
        
    return real_idx_for_image, real_ts_for_image

def get_start_end_frames(gaze_csv, real_images_idx):

    all_frames = [int(f) for f in get_csv_column(gaze_csv, WORLD_IDX, 1)]
    start_frames = []
    end_frames = []

    for i in real_images_idx:
        start_frame = all_frames[(i+1)] + 10
        while (not start_frame in all_frames):
            start_frame+=1
        start_frames.append(start_frame)

        end_frame = start_frame + 35
        while (not end_frame in all_frames):
            end_frame+=1
        end_frames.append(end_frame)

    return start_frames, end_frames
    
def get_start_end_frames_indices(gaze_csv, start_frames, end_frames):

    all_frames = [int(f) for f in get_csv_column(gaze_csv, WORLD_IDX, 1)]
    start_frames_indices = []
    end_frames_indices = []

    for start_frame in start_frames:
        start_frame_idx = all_frames.index(start_frame)
        start_frames_indices.append(start_frame_idx)

    for end_frame in end_frames:
        end_frame_idx = all_frames.index(end_frame)
        
        while all_frames[end_frame_idx] == end_frame:
            end_frame_idx+=1

        end_frames_indices.append(end_frame_idx)

    return start_frames_indices, end_frames_indices

def make_norm_XY_list(gaze_csv, start_frames_indices, end_frames_indices):

    if (len(start_frames_indices) != len(end_frames_indices)):
        return []
    
    num_pictures = len(start_frames_indices)
    norm_X = get_csv_column(gaze_csv, NORM_POS_X, 1)
    norm_Y = get_csv_column(gaze_csv, NORM_POS_Y, 1)
    norm_XY_list = []

    for i in range(num_pictures):
        start_idx = start_frames_indices[i]
        end_idx = end_frames_indices[i]
        sub_norm_X = norm_X[start_idx:end_idx]
        sub_norm_Y = norm_Y[start_idx:end_idx]
        norm_XY_list.append([sub_norm_X, sub_norm_Y])

    return norm_XY_list

def createNormCSV(part_list):

    for part in part_list:

        print(part)

        events_csv = "./parts/breakItDown/" + part + "/Events.csv"
        gaze_csv = "./parts/breakItDown/" + part + "/gaze_positions.csv"
        # debug CSVs
        deltas_csv = "./parts/breakItDown/" + part + "/deltas.csv"
        norms_csv = "./parts/breakItDown/" + part + "/norms.csv"
        # output CSV
        normXY_csv = "./parts/breakItDown/" + part + "/normXY.csv"

        if not os.path.exists(deltas_csv):
                Path(deltas_csv).touch()
        if not os.path.exists(norms_csv):
                Path(norms_csv).touch()
        if not os.path.exists(normXY_csv):
                Path(normXY_csv).touch()

        image_list = get_images_order(events_csv)
        event_ts = get_events_timestamps(events_csv)
        deltas = calc_deltas(event_ts)

        first_gaze_ts = float(gaze_dict[part])
        images_ts = [delta + first_gaze_ts for delta in deltas]
        real_images_idx, real_images_ts = calc_real_ts_for_image(gaze_csv, images_ts)

        start_frames, end_frames = get_start_end_frames(gaze_csv, real_images_idx)
        start_frames_indices, end_frames_indices = get_start_end_frames_indices(gaze_csv, start_frames, end_frames)
        list_of_norm_xy = make_norm_XY_list(gaze_csv, start_frames_indices, end_frames_indices)

        rows = zip(image_list, event_ts, deltas, images_ts, real_images_idx, real_images_ts)
        write_lists_to_csv(deltas_csv, rows)
        rows = zip(image_list, real_images_idx, start_frames, end_frames, start_frames_indices, end_frames_indices)#, list_of_norm_xy)
        write_lists_to_csv(norms_csv, rows)
        rows = zip(image_list, list_of_norm_xy)
        write_lists_to_csv(normXY_csv, rows) 

part_list = ["AR",  "ASH", "AT",  "DI",  "DM",  "DZ",
             "EL",  "EMZ", "HCH", "IL",  "IS",  "MA",
             "MBA", "NC",  "NCH", "NN",  "NS",  "NT", 
             "OA",  "OB",  "OR",  "RM",  "RN",  "SB", 
             "SHB", "SHL", "SHM", "SHO", "SHS", "SSW",
             "TCH", "TLZ", "YCH", "YH",  
             "YK",  "YL",  "YLA", "YM",  "YOB", "YUC"]
part_list = ["SHL"]
  
createNormCSV(part_list)
